<a href="https://colab.research.google.com/github/selimctkl/freshman/blob/main/Duygu_Tan%C4%B1ma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from keras import backend as K
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import tensorflow as tf

In [ ]:
data = pd.read_csv("fer2013.csv")

In [ ]:
data.shape

In [ ]:
data["Usage"].value_counts()

In [ ]:
data.head()

In [ ]:
train_data = data[data.Usage == "Training"]

In [ ]:
train_data

In [ ]:
train_pixels = train_data.pixels.str.split(" ").tolist()

train_pixels = pd.DataFrame(train_pixels, dtype = int)
train_images = train_pixels.values
train_images = train_images.astype(np.float)


In [ ]:
def show(img):
    show_image = img.reshape((48,48))
    plt.axis("off")
    plt.imshow(show_image, cmap = "gray")

In [ ]:
show(train_images[7])

In [ ]:
train_labels_flat = train_data["emotion"].values.ravel()
train_labels_count = np.unique(train_labels_flat).shape[0]
print("Farklı Yüz İfadeleri sayısı %d" %train_labels_count)

In [ ]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [ ]:
y_train = dense_to_one_hot(train_labels_flat, train_labels_count)
y_train = y_train.astype(np.uint8)
print(y_train.shape)

In [ ]:
np.unique(data["Usage"].values.ravel())
print("test veri seti örnek sayı %d" %(len(data[data.Usage == "PublicTest"])))

In [ ]:
test_data = data[data.Usage == "PublicTest"]
test_pixels = test_data.pixels.str.split(" ").tolist()
test_pixels = pd.DataFrame(test_pixels, dtype=int)
test_images = test_pixels.values
test_images = test_images.astype(np.float)
print(test_images)
      

In [ ]:
show(test_images[6])

In [ ]:
test_labels_flat = test_data["emotion"].values.ravel()
test_labels_count = np.unique(test_labels_flat).shape[0]

y_test = dense_to_one_hot(test_labels_flat, test_labels_count)

print(y_test.shape)

In [ ]:
plt.figure(0, figsize=(12,6))
for i in range(1,13):
    plt.subplot(3,4,i)
    plt.axis("off")

    image = test_images[i].reshape(48,48)
    plt.imshow(image, cmap="gray")

plt.tight_layout( pad =1 )
plt.show()

In [ ]:
from keras.layers.core.activation import Activation
Activation,
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
model = Sequential()

# katman 1

model.add(Conv2D(64, 3, data_format = "channels_last", kernel_initializer= "he_normal", input_shape = (48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))

# katman 2

model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.6))

# katman 3

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

# katman 4

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

# katman 5

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2), strides=2))
model.add(Dropout(0.6))

# tam bağlantı

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.6))

# çıktı katman

model.add(Dense(7))
model.add(Activation("softmax"))
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()


In [ ]:
x_train = train_images.reshape(-1, 48, 48, 1)
x_test = test_images.reshape(-1, 48, 48, 1)

print("Train:", x_train.shape)
print("Test", x_test.shape)

In [ ]:
print("Train:", y_train.shape)
print("Test:", y_test.shape)


In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath =  "data/face_model.h5", verbose = 1, save_best_only = True)
epochs = 10
batchSize = 100
hist = model.fit(x_train, y_train,
                 epochs = epochs,
                 shuffle = True,
                 batch_size = batchSize,
                 validation_data =(x_test, y_test),
                 callbacks = [checkpointer], verbose = 2)



In [ ]:
plt.figure(figsize = (14, 3))
plt.subplot(1, 2, 1)
plt.suptitle("Eğitim", fontsize = 10)
plt.ylabel("Loss", fontsize= 16)
plt.plot(hist.history["loss"], color = "b", label = "Training Loss")
plt.plot(hist.history["val_loss"], color = "r", label = "validation Loss")
plt.legend(loc = "upper right")

plt.subplot(1, 2, 2)
plt.suptitle("Accuracy", fontsize = 16)
plt.plot(hist.history["accuracy"], color = "b", label = "Training Accuracy")
plt.plot(hist.history["val_accuracy"], color = "r", label = "validation Accuracy")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
test = data[["emotion", "pixels"]][data["Usage"] == "PrivateTest"]
test["pixels"] = test["pixels"].apply(lambda im: np.fromstring(im, sep=" "))
test.head()

In [ ]:
x_test_private = np.vstack(test["pixels"].values)
y_test_private = np.array(test["emotion"])

In [ ]:
pip install np_utils
from keras.utils.np_utils import to_categorical

In [ ]:
from keras import utils

In [ ]:
x_test_private = x_test_private.reshape(-1, 48, 48, 1)
y_test_private = utils.to_categorical(y_test_private)
x_test_private.shape, y_test_private.shape 

In [ ]:
score = model.evaluate(x_test_private, y_test_private, verbose = 0)
print("PrivateTest üzerinde doğruluk başarımı:" , score)

In [ ]:

img = keras.utils.image_utils.load_img("biometric.JPG", grayscale=True, target_size=(48, 48))

x = keras.utils.image_utils.img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
#Duygu Analizi(custom[0])


#1
objects = ('kızgın', 'nefret', 'korku', 'mutlu', 'üzgün', 'şaşırma', 'doğal')
y_pos = np.arange(len(objects))
    
plt.bar(y_pos, custom[0], align='center', alpha=0.5, color='g')
plt.xticks(y_pos, objects)
plt.ylabel('yüzde')
plt.title('duygu')
plt.show()

#2
x = np.array(x, 'float32')
x = x.reshape([48, 48]);
plt.axis('off')
plt.gray()
plt.imshow(img)

plt.show()
#----------